In [52]:
import multiprocessing
from multiprocessing import Pool
... # you wrote a lot of code here ...
from shapely.geometry import Point
from geopandas.tools import sjoin
from pyproj import Proj, transform
import pandas as pd
import geopandas as gpd

In [59]:
tpl_crs = Proj({'proj': 'aea',
                            'lat_1': 20,
                    'lat_2': 60,
                    'lat_0': 40,
                    'lon_0': -96,
                    'x_0': 0,
                    'y_0': 0,
                    'datum': 'NAD83',
                    'units': 'm',
                    'no_defs': True,
                    'wktext': True})
    
def reproj_coords(coords):
    tweet_crs = Proj({'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'WGS84'})
    tpl_crs = Proj({'proj': 'aea',
                            'lat_1': 20,
                    'lat_2': 60,
                    'lat_0': 40,
                    'lon_0': -96,
                    'x_0': 0,
                    'y_0': 0,
                    'datum': 'NAD83',
                    'units': 'm',
                    'no_defs': True,
                    'wktext': True})
    lon = coords[0]
    lat = coords[1]
    lon, lat = transform(tweet_crs, tpl_crs, lon, lat)
    return Point(lon, lat)

In [12]:
# Load tweets to dataframe
tweet_file = "../data/raw/tweets/OH_Cleveland_3916000.json"
tweet_df = pd.read_json(tweet_file)
try:
    tweet_df.drop("fastText_lang", axis=1, inplace=True)
except KeyError:
    pass
print("Read {} tweets from file".format(len(tweet_df)))

Read 525547 tweets from file


In [45]:
test_geo = [x['coordinates'] for x in tweet_df.geo.values]
test_geo_rp = [reproj_coords(x) for x in test_geo]

In [47]:
%%time
with Pool(None) as p:
    tweet_reproj = p.map(reproj_coords, test_geo)

CPU times: user 6.01 s, sys: 753 ms, total: 6.76 s
Wall time: 1min 7s


In [48]:
# %%time
tweet_geometry = [reproj_coords(xy) for xy in test_geo]
#tweet_df = tweet_df.drop('geo', axis=1)



CPU times: user 59.6 s, sys: 31.1 s, total: 1min 30s
Wall time: 1min 35s


In [60]:
# convert to geodataframe, with reprojeted geometry
#tweet_gdf = gpd.GeoDataFrame(tweet_df, crs= gdf.crs, geometry= tweet_geometry)
#print("Tweets reprojected {}".format(gdf.crs))
#tweet_gdf = gpd.GeoDataFrame(tweet_df, crs= tpl_crs, geometry= tweet_reproj)#
#tweet_gdf.head()
#print("Tweets reprojected {}".format(gdf.crs))

,id,pure_text,tweet_created_at,geometry
0,587044015549349900,Go Tribe! - Drinking a Game Of Thrones - Valar...,2015-04-12 00:06:14,POINT (1119240.770515308 258978.4993788982)
1,587044124735471600,@hiselenuh @dontshootskrlx @mauricemorfaw ever...,2015-04-12 00:06:40,POINT (1121988.853604449 262577.5272637045)
2,587044703247409200,Outtake from an earlier shot in the library. #...,2015-04-12 00:08:58,POINT (1119080.94305365 261818.4123514606)
3,587046028601229300,Britches kill me getting pregnant and ain't go...,2015-04-12 00:14:14,POINT (1125305.19637322 266770.4447996031)
4,587046066501062700,I just want a significant other 😕,2015-04-12 00:14:23,POINT (1127078.498761543 257083.5891447938)
